<a href="https://colab.research.google.com/github/valin1/rllib-tutorial/blob/master/RLlib_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Dependencies

If you are running on Google Colab, you need to install the necessary dependencies before beginning the exercise.

In [0]:
print('NOTE: Intentionally crashing session to use the newly installed library.\n')

!pip uninstall -y pyarrow
!pip install ray[debug]==0.7.5
!pip install bs4

!git clone https://github.com/ray-project/tutorial || true
from tutorial.rllib_exercises import test_exercises

print("Successfully installed all the dependencies!")

# A hack to force the runtime to restart, needed to include the above dependencies.
import os
os._exit(0)

NOTE: Intentionally crashing session to use the newly installed library.

Uninstalling pyarrow-0.14.1:
  Successfully uninstalled pyarrow-0.14.1
     |████████████████████████████████| 74.9MB 1.2MB/s 
     |████████████████████████████████| 71kB 20.4MB/s 
     |████████████████████████████████| 1.3MB 33.5MB/s 
     |████████████████████████████████| 2.8MB 51.5MB/s 
  Created wheel for setproctitle: filename=setproctitle-1.1.10-cp36-cp36m-linux_x86_64.whl size=33936 sha256=b797e42ddd49d22bd5cd060b96c754e1ca8c4104d6347c9ece49e8d1464a466f
  Stored in directory: /root/.cache/pip/wheels/e6/b1/a6/9719530228e258eba904501fef99d5d85c80d52bd8f14438a3
Successfully built setproctitle
  Found existing installation: protobuf 3.7.1
    Uninstalling protobuf-3.7.1:
      Successfully uninstalled protobuf-3.7.1


Cloning into 'tutorial'...
remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 884 (delta 16), reused 14 (delta 6), pack-reused 848
Receiving objects: 100% (884/884), 31.13 MiB | 22.95 MiB/s, done.
Resolving deltas: 100% (481/481), done.


# RL Exercise - Markov Decision Processes

**GOAL:** The goal of the exercise is to introduce the Markov Decision Process abstraction and to show how to use Markov Decision Processes in Python.

**The key abstraction in reinforcement learning is the Markov decision process (MDP).** An MDP models sequential interactions with an external environment. It consists of the following:
- a **state space**
- a set of **actions**
- a **transition function** which describes the probability of being in a state $s'$ at time $t+1$ given that the MDP was in state $s$ at time $t$ and action $a$ was taken
- a **reward function**, which determines the reward received at time $t$
- a **discount factor** $\gamma$

More details are available [here](https://en.wikipedia.org/wiki/Markov_decision_process).

**NOTE:** Reinforcement learning algorithms are often applied to problems that don't strictly fit into the MDP framework. In particular, situations in which the state of the environment is not fully observed lead to violations of the MDP assumption. Nevertheless, RL algorithms can be applied anyway.

## Policies

A **policy** is a function that takes in a **state** and returns an **action**. A policy may be stochastic (i.e., it may sample from a probability distribution) or it can be deterministic.

The **goal of reinforcement learning** is to learn a **policy** for maximizing the cumulative reward in an MDP. That is, we wish to find a policy $\pi$ which solves the following optimization problem

\begin{equation}
\arg\max_{\pi} \sum_{t=1}^T \gamma^t R_t(\pi),
\end{equation}

where $T$ is the number of steps taken in the MDP (this is a random variable and may depend on $\pi$) and $R_t$ is the reward received at time $t$ (also a random variable which depends on $\pi$).

A number of algorithms are available for solving reinforcement learning problems. Several of the most widely known are [value iteration](https://en.wikipedia.org/wiki/Markov_decision_process#Value_iteration), [policy iteration](https://en.wikipedia.org/wiki/Markov_decision_process#Policy_iteration), and [Q learning](https://en.wikipedia.org/wiki/Q-learning).

## RL in Python

The `gym` Python module provides MDP interfaces to a variety of simulators. For example, the CartPole environment interfaces with a simple simulator which simulates the physics of balancing a pole on a cart. The CartPole problem is described at https://gym.openai.com/envs/CartPole-v0. This example fits into the MDP framework as follows.
- The **state** consists of the position and velocity of the cart as well as the angle and angular velocity of the pole that is balancing on the cart.
- The **actions** are to decrease or increase the cart's velocity by one unit.
- The **transition function** is deterministic and is determined by simulating physical laws.
- The **reward function** is a constant 1 as long as the pole is upright, and 0 once the pole has fallen over. Therefore, maximizing the reward means balancing the pole for as long as possible.
- The **discount factor** in this case can be taken to be 1.

More information about the `gym` Python module is available at https://gym.openai.com/.

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import gym
import numpy as np

The code below illustrates how to create and manipulate MDPs in Python. An MDP can be created by calling `gym.make`. Gym environments are identified by names like `CartPole-v0`. A **catalog of built-in environments** can be found at https://gym.openai.com/envs.

In [0]:
env = gym.make('CartPole-v0')
print('Created env:', env)

Created env: <TimeLimit<CartPoleEnv<CartPole-v0>>>


Reset the state of the MDP by calling `env.reset()`. This call returns the initial state of the MDP.

In [0]:
state = env.reset()
print('The starting state is:', state)

The starting state is: [ 0.03192058  0.03550393 -0.04973072 -0.0396154 ]


The `env.step` method takes an action (in the case of the CartPole environment, the appropriate actions are 0 or 1, for moving left or right). It returns a tuple of four things:
1. the new state of the environment
2. a reward
3. a boolean indicating whether the simulation has finished
4. a dictionary of miscellaneous extra information

In [0]:
# Simulate taking an action in the environment. Appropriate actions for
# the CartPole environment are 0 and 1 (for moving left and right).
action = 0
state, reward, done, info = env.step(action)
print(state, reward, done, info)

[ 0.03263066 -0.15887092 -0.05052303  0.2369716 ] 1.0 False {}


A **rollout** is a simulation of a policy in an environment. It alternates between choosing actions based (using some policy) and taking those actions in the environment.

The code below performs a rollout in a given environment. It takes **random actions** until the simulation has finished and returns the cumulative reward.

In [0]:
def random_rollout(env):
    state = env.reset()
    
    done = False
    cumulative_reward = 0

    # Keep looping as long as the simulation has not finished.
    while not done:
        # Choose a random action (either 0 or 1).
        action = np.random.choice([0, 1])
        
        # Take the action in the environment.
        state, reward, done, _ = env.step(action)
        
        # Update the cumulative reward.
        cumulative_reward += reward
    
    # Return the cumulative reward.
    return cumulative_reward
    
reward = random_rollout(env)
print(reward)
reward = random_rollout(env)
print(reward)

16.0
24.0


**EXERCISE:** Finish implementing the `rollout_policy` function below, which should take an environment *and* a policy. The *policy* is a function that takes in a *state* and returns an *action*. The main difference is that instead of choosing a **random action**, the action should be chosen **with the policy** (as a function of the state).

In [0]:
def rollout_policy(env, policy):
    state = env.reset()
    
    done = False
    cumulative_reward = 0

    # EXERCISE: Fill out this function by copying the 'random_rollout' function
    # and then modifying it to choose the action using the policy.
    # Keep looping as long as the simulation has not finished.
    while not done:
        # Choose a random action (either 0 or 1).
        action = policy(state)
        
        # Take the action in the environment.
        state, reward, done, _ = env.step(action)
        
        # Update the cumulative reward.
        cumulative_reward += reward

    # Return the cumulative reward.
    return cumulative_reward

def sample_policy1(state):
    return 0 if state[0] < 0 else 1

def sample_policy2(state):
    return 1 if state[0] < 0 else 0

reward1 = np.mean([rollout_policy(env, sample_policy1) for _ in range(100)])
reward2 = np.mean([rollout_policy(env, sample_policy2) for _ in range(100)])

print('The first sample policy got an average reward of {}.'.format(reward1))
print('The second sample policy got an average reward of {}.'.format(reward2))

assert 5 < reward1 < 15, ('Make sure that rollout_policy computes the action '
                          'by applying the policy to the state.')
assert 25 < reward2 < 35, ('Make sure that rollout_policy computes the action '
                           'by applying the policy to the state.')

The first sample policy got an average reward of 9.4.
The second sample policy got an average reward of 30.3.


# RLlib Exercise 1 - Proximal Policy Optimization

**GOAL:** The goal of this exercise is to demonstrate how to use the proximal policy optimization (PPO) algorithm.

To understand how to use **RLlib**, see the documentation at http://rllib.io.

PPO is described in detail in https://arxiv.org/abs/1707.06347. It is a variant of Trust Region Policy Optimization (TRPO) described in https://arxiv.org/abs/1502.05477

PPO works in two phases. In one phase, a large number of rollouts are performed (in parallel). The rollouts are then aggregated on the driver and a surrogate optimization objective is defined based on those rollouts. We then use SGD to find the policy that maximizes that objective with a penalty term for diverging too much from the current policy.

![ppo](https://raw.githubusercontent.com/ucbrise/risecamp/risecamp2018/ray/tutorial/rllib_exercises/ppo.png)

**NOTE:** The SGD optimization step is best performed in a data-parallel manner over multiple GPUs. This is exposed through the `num_gpus` field of the `config` dictionary (for this to work, you must be using a machine that has GPUs).

In [0]:
# Be sure to install the latest version of RLlib.
! pip install -U ray[rllib]

Requirement already up-to-date: ray[rllib] in /usr/local/lib/python3.6/dist-packages (0.7.5)
     |████████████████████████████████| 399kB 9.1MB/s 
     |████████████████████████████████| 22.1MB 43.0MB/s 


In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import gym
import ray
from ray.rllib.agents.ppo import PPOTrainer, DEFAULT_CONFIG
from ray.tune.logger import pretty_print

In [0]:
# Start up Ray. This must be done before we instantiate any RL agents.
ray.init(num_cpus=3, ignore_reinit_error=True, log_to_driver=False)

2019-10-17 17:54:06,994	INFO resource_spec.py:205 -- Starting Ray with 6.05 GiB memory available for workers and up to 3.05 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).


{'node_ip_address': '172.28.0.2',
 'object_store_address': '/tmp/ray/session_2019-10-17_17-54-06_993248_226/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2019-10-17_17-54-06_993248_226/sockets/raylet',
 'redis_address': '172.28.0.2:25628',
 'session_dir': '/tmp/ray/session_2019-10-17_17-54-06_993248_226',
 'webui_url': None}

Instantiate a PPOTrainer object. We pass in a config object that specifies how the network and training procedure should be configured. Some of the parameters are the following.

- `num_workers` is the number of actors that the agent will create. This determines the degree of parallelism that will be used.
- `num_sgd_iter` is the number of epochs of SGD (passes through the data) that will be used to optimize the PPO surrogate objective at each iteration of PPO.
- `sgd_minibatch_size` is the SGD batch size that will be used to optimize the PPO surrogate objective.
- `model` contains a dictionary of parameters describing the neural net used to parameterize the policy. The `fcnet_hiddens` parameter is a list of the sizes of the hidden layers.

In [0]:
config = DEFAULT_CONFIG.copy()
config['num_workers'] = 1
config['num_sgd_iter'] = 30
config['sgd_minibatch_size'] = 128
config['model']['fcnet_hiddens'] = [100, 100]
config['num_cpus_per_worker'] = 0  # This avoids running out of resources in the notebook environment when this cell is re-executed

agent = PPOTrainer(config, 'CartPole-v0')

2019-10-17 17:54:39,295	INFO trainer.py:344 -- Tip: set 'eager': true or the --eager flag to enable TensorFlow eager execution
2019-10-17 17:54:40,973	INFO rollout_worker.py:768 -- Built policy map: {'default_policy': <ray.rllib.policy.tf_policy_template.PPOTFPolicy object at 0x7fea5c3df400>}
2019-10-17 17:54:40,975	INFO rollout_worker.py:769 -- Built preprocessor map: {'default_policy': <ray.rllib.models.preprocessors.NoPreprocessor object at 0x7fea5c3df160>}
2019-10-17 17:54:40,976	INFO rollout_worker.py:370 -- Built filter map: {'default_policy': <ray.rllib.utils.filter.NoFilter object at 0x7feb23f55828>}
2019-10-17 17:54:41,009	INFO multi_gpu_optimizer.py:93 -- LocalMultiGPUOptimizer devices ['/cpu:0']
2019-10-17 17:54:45,150	WARNING util.py:45 -- Install gputil for GPU system monitoring.


Train the policy on the `CartPole-v0` environment for 2 steps. The CartPole problem is described at https://gym.openai.com/envs/CartPole-v0.

**EXERCISE:** Inspect how well the policy is doing by looking for the lines that say something like

```
episode_len_mean: 22.262569832402235
episode_reward_mean: 22.262569832402235
```

This indicates how much reward the policy is receiving and how many time steps of the environment the policy ran. The maximum possible reward for this problem is 200. The reward and trajectory length are very close because the agent receives a reward of one for every time step that it survives (however, that is specific to this environment).

In [0]:
for i in range(2):
    result = agent.train()
    print(pretty_print(result))

2019-10-17 17:55:42,250	INFO tf_policy.py:358 -- Optimizing variable <tf.Variable 'default_policy/fc_1/kernel:0' shape=(4, 100) dtype=float32>
2019-10-17 17:55:42,255	INFO tf_policy.py:358 -- Optimizing variable <tf.Variable 'default_policy/fc_1/bias:0' shape=(100,) dtype=float32>
2019-10-17 17:55:42,256	INFO tf_policy.py:358 -- Optimizing variable <tf.Variable 'default_policy/fc_value_1/kernel:0' shape=(4, 100) dtype=float32>
2019-10-17 17:55:42,257	INFO tf_policy.py:358 -- Optimizing variable <tf.Variable 'default_policy/fc_value_1/bias:0' shape=(100,) dtype=float32>
2019-10-17 17:55:42,260	INFO tf_policy.py:358 -- Optimizing variable <tf.Variable 'default_policy/fc_2/kernel:0' shape=(100, 100) dtype=float32>
2019-10-17 17:55:42,262	INFO tf_policy.py:358 -- Optimizing variable <tf.Variable 'default_policy/fc_2/bias:0' shape=(100,) dtype=float32>
2019-10-17 17:55:42,264	INFO tf_policy.py:358 -- Optimizing variable <tf.Variable 'default_policy/fc_value_2/kernel:0' shape=(100, 100) dtyp

custom_metrics: {}
date: 2019-10-17_17-55-47
done: false
episode_len_mean: 22.08839779005525
episode_reward_max: 91.0
episode_reward_mean: 22.08839779005525
episode_reward_min: 9.0
episodes_this_iter: 181
episodes_total: 181
experiment_id: 72b87ab642c04632b54c6f2674cec748
hostname: dfdd5ae0844a
info:
  grad_time_ms: 4964.206
  learner:
    default_policy:
      cur_kl_coeff: 0.20000000298023224
      cur_lr: 4.999999873689376e-05
      entropy: 0.6622359156608582
      entropy_coeff: 0.0
      kl: 0.030930981040000916
      policy_loss: -0.04017391800880432
      total_loss: 177.03761291503906
      vf_explained_var: 0.02820538729429245
      vf_loss: 177.07167053222656
  load_time_ms: 130.711
  num_steps_sampled: 4000
  num_steps_trained: 3968
  sample_time_ms: 5657.124
  update_time_ms: 1293.891
iterations_since_restore: 1
node_ip: 172.28.0.2
num_healthy_workers: 1
off_policy_estimator: {}
perf:
  cpu_util_percent: 18.998876404494382
  ram_util_percent: 11.026966292134833
pid: 226
po

**EXERCISE:** The current network and training configuration are too large and heavy-duty for a simple problem like CartPole. Modify the configuration to use a smaller network and to speed up the optimization of the surrogate objective (fewer SGD iterations and a larger batch size should help).

In [0]:
config = DEFAULT_CONFIG.copy()
config['num_workers'] = 3
config['num_sgd_iter'] = 15
config['sgd_minibatch_size'] = 500
config['model']['fcnet_hiddens'] = [100, 100]
config['num_cpus_per_worker'] = 0

agent = PPOTrainer(config, 'CartPole-v0')

2019-10-17 17:56:50,174	INFO trainer.py:344 -- Tip: set 'eager': true or the --eager flag to enable TensorFlow eager execution
2019-10-17 17:56:51,568	INFO rollout_worker.py:768 -- Built policy map: {'default_policy': <ray.rllib.policy.tf_policy_template.PPOTFPolicy object at 0x7fea4c86e6d8>}
2019-10-17 17:56:51,576	INFO rollout_worker.py:769 -- Built preprocessor map: {'default_policy': <ray.rllib.models.preprocessors.NoPreprocessor object at 0x7fea4c8ddc50>}
2019-10-17 17:56:51,577	INFO rollout_worker.py:370 -- Built filter map: {'default_policy': <ray.rllib.utils.filter.NoFilter object at 0x7fea4c86ebe0>}
2019-10-17 17:56:51,667	INFO multi_gpu_optimizer.py:93 -- LocalMultiGPUOptimizer devices ['/cpu:0']
2019-10-17 17:57:02,494	INFO trainable.py:102 -- _setup took 12.290 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2019-10-17 17:57:02,505	WARNING util.py:45 -- Install gputil for GPU system monitoring.


**EXERCISE:** Train the agent and try to get a reward of 200. If it's training too slowly you may need to modify the config above to use fewer hidden units, a larger `sgd_minibatch_size`, a smaller `num_sgd_iter`, or a larger `num_workers`.

This should take around 20 or 30 training iterations.

In [0]:
for i in range(2):
    result = agent.train()
    print(pretty_print(result))

custom_metrics: {}
date: 2019-10-17_17-57-18
done: false
episode_len_mean: 21.87292817679558
episode_reward_max: 67.0
episode_reward_mean: 21.87292817679558
episode_reward_min: 8.0
episodes_this_iter: 181
episodes_total: 181
experiment_id: b4f3442f90b64f319adc97c1ab89487e
hostname: dfdd5ae0844a
info:
  grad_time_ms: 1453.591
  learner:
    default_policy:
      cur_kl_coeff: 0.20000000298023224
      cur_lr: 4.999999873689376e-05
      entropy: 0.6715409159660339
      entropy_coeff: 0.0
      kl: 0.022996434941887856
      policy_loss: -0.03003411367535591
      total_loss: 247.77706909179688
      vf_explained_var: 1.2256205081939697e-05
      vf_loss: 247.802490234375
  load_time_ms: 103.759
  num_steps_sampled: 4000
  num_steps_trained: 4000
  sample_time_ms: 4126.344
  update_time_ms: 1295.654
iterations_since_restore: 1
node_ip: 172.28.0.2
num_healthy_workers: 3
off_policy_estimator: {}
perf:
  cpu_util_percent: 61.1
  ram_util_percent: 15.208695652173919
pid: 226
policy_reward_m

Checkpoint the current model. The call to `agent.save()` returns the path to the checkpointed model and can be used later to restore the model.

In [0]:
checkpoint_path = agent.save()
print(checkpoint_path)

Now let's use the trained policy to make predictions.

**NOTE:** Here we are loading the trained policy in the same process, but in practice, this would often be done in a different process (probably on a different machine).

In [0]:
trained_config = config.copy()

test_agent = PPOTrainer(trained_config, 'CartPole-v0')
test_agent.restore(checkpoint_path)

Now use the trained policy to act in an environment. The key line is the call to `test_agent.compute_action(state)` which uses the trained policy to choose an action.

**EXERCISE:** Verify that the reward received roughly matches up with the reward printed in the training logs.

In [0]:
env = gym.make('CartPole-v0')
state = env.reset()
done = False
cumulative_reward = 0

while not done:
    action = test_agent.compute_action(state)
    state, reward, done, _ = env.step(action)
    cumulative_reward += reward

print(cumulative_reward)

# RLlib Exercise 2 - Custom Environments and Reward Shaping

**GOAL:** The goal of this exercise is to demonstrate how to adapt your own problem to use RLlib.

To understand how to use **RLlib**, see the documentation at http://rllib.io.

RLlib is not only easy to use in simulated benchmarks but also in the real-world. Here, we will cover two important concepts: how to create your own Markov Decision Process abstraction, and how to shape the reward of your environment so make your agent more effective. 

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import gym
from gym import spaces
import numpy as np
from tutorial.rllib_exercises import test_exercises

import ray
from ray.rllib.agents.ppo import PPOTrainer, DEFAULT_CONFIG

ray.init(ignore_reinit_error=True, log_to_driver=False)

## 1. Different Spaces

The first thing to do when formulating an RL problem is to specify the dimensions of your observation space and action space. Abstractions for these are provided in ``gym``. 

### **Exercise 1:** Match different actions to their corresponding space.

The purpose of this exercise is to familiarize you with different Gym spaces. For example:

    discrete = spaces.Discrete(10)
    print("Random sample of this space: ", [discrete.sample() for i in range(4)])

Use `help(spaces)` or `help([specific space])` (i.e., `help(spaces.Discrete)`) for more info.

In [0]:
action_space_map = {
    "discrete_10": spaces.Discrete(10),
    "box_1": spaces.Box(0, 1, shape=(1,)),
    "box_3x1": spaces.Box(-2, 2, shape=(3, 1)),
    "multi_discrete": spaces.MultiDiscrete([ 5, 2, 2, 4 ])
}

action_space_jumble = {
    "discrete_10": 1,
    "CHANGE_ME": np.array([0, 0, 0, 2]),
    "CHANGE_ME": np.array([[-1.2657754], [-1.6528835], [ 0.5982418]]),
    "CHANGE_ME": np.array([0.89089584]),
}


for space_id, state in action_space_jumble.items():
    assert action_space_map[space_id].contains(state), (
        "Looks like {} to {} is matched incorrectly.".format(space_id, state))
    
print("Success!")

## **Exercise 2**: Setting up a custom environment with rewards

We'll setup an `n-Chain` environment, which presents moves along a linear chain of states, with two actions:

     (0) forward, which moves along the chain but returns no reward
     (1) backward, which returns to the beginning and has a small reward

The end of the chain, however, presents a large reward, and by moving 'forward', at the end of the chain this large reward can be repeated.

#### Step 1: Implement ``ChainEnv._setup_spaces``

We'll use a `spaces.Discrete` action space and observation space. Implement `ChainEnv._setup_spaces` so that `self.action_space` and `self.obseration_space` are proper gym spaces.
  
1. Observation space is an integer in ``[0 to n-1]``.
2. Action space is an integer in ``[0, 1]``.

For example:

```python
    self.action_space = spaces.Discrete(2)
    self.observation_space = ...
```

You should see a message indicating tests passing when done correctly!

#### Step 2: Implement a reward function.

When `env.step` is called, it returns a tuple of ``(state, reward, done, info)``. Right now, the reward is always 0. 

Implement it so that 

1. ``action == 1`` will return `self.small_reward`.
2. ``action == 0`` will return 0 if `self.state < self.n - 1`.
3. ``action == 0`` will return `self.large_reward` if `self.state == self.n - 1`.

You should see a message indicating tests passing when done correctly. 

In [0]:
class ChainEnv(gym.Env):
    
    def __init__(self, env_config = None):
        env_config = env_config or {}
        self.n = env_config.get("n", 20)
        self.small_reward = env_config.get("small", 2)  # payout for 'backwards' action
        self.large_reward = env_config.get("large", 10)  # payout at end of chain for 'forwards' action
        self.state = 0  # Start at beginning of the chain
        self._horizon = self.n
        self._counter = 0  # For terminating the episode
        self._setup_spaces()
    
    def _setup_spaces(self):
        ##############
        # TODO: Implement this so that it passes tests
        self.action_space = None
        self.observation_space = None
        ##############

    def step(self, action):
        assert self.action_space.contains(action)
        if action == 1:  # 'backwards': go back to the beginning, get small reward
            ##############
            # TODO 2: Implement this so that it passes tests
            reward = -1
            ##############
            self.state = 0
        elif self.state < self.n - 1:  # 'forwards': go up along the chain
            ##############
            # TODO 2: Implement this so that it passes tests
            reward = -1
            self.state += 1
        else:  # 'forwards': stay at the end of the chain, collect large reward
            ##############
            # TODO 2: Implement this so that it passes tests
            reward = -1
            ##############
        self._counter += 1
        done = self._counter >= self._horizon
        return self.state, reward, done, {}

    def reset(self):
        self.state = 0
        self._counter = 0
        return self.state
    
# Tests here:
test_exercises.test_chain_env_spaces(ChainEnv)
test_exercises.test_chain_env_reward(ChainEnv)

### Now let's train a policy on the environment and evaluate this policy on our environment.

You'll see that despite an extremely high reward, the policy has barely explored the state space.

In [0]:
trainer_config = DEFAULT_CONFIG.copy()
trainer_config['num_workers'] = 1
trainer_config["train_batch_size"] = 400
trainer_config["sgd_minibatch_size"] = 64
trainer_config["num_sgd_iter"] = 10

In [0]:
trainer = PPOTrainer(trainer_config, ChainEnv);
for i in range(20):
    print("Training iteration {}...".format(i))
    trainer.train()

In [0]:
env = ChainEnv({})
state = env.reset()

done = False
max_state = -1
cumulative_reward = 0

while not done:
    action = trainer.compute_action(state)
    state, reward, done, results = env.step(action)
    max_state = max(max_state, state)
    cumulative_reward += reward

print("Cumulative reward you've received is: {}. Congratulations!".format(cumulative_reward))
print("Max state you've visited is: {}. This is out of {} states.".format(max_state, env.n))

## Exercise 3: Shaping the reward to encourage proper behavior.

You'll see that despite an extremely high reward, the policy has barely explored the state space. This is often the situation - where the reward designed to encourage a particular solution is suboptimal, and the behavior created is unintended.

#### Modify `ShapedChainEnv.step` to provide a reward that encourages the policy to traverse the chain (not just stick to 0). Do not change the behavior of the environment (the action -> state behavior should be the same).

You can change the reward to be whatever you wish.

In [0]:
class ShapedChainEnv(ChainEnv):
    def step(self, action):
        assert self.action_space.contains(action)
        if action == 1:  # 'backwards': go back to the beginning
            reward = -1
            self.state = 0
        elif self.state < self.n - 1:  # 'forwards': go up along the chain
            reward = -1
            self.state += 1
        else:  # 'forwards': stay at the end of the chain
            reward = -1
        self._counter += 1
        done = self._counter >= self._horizon
        return self.state, reward, done, {}
    
test_exercises.test_chain_env_behavior(ShapedChainEnv)

### Evaluate `ShapedChainEnv` by running the cell below.

This trains PPO on the new env and counts the number of states seen.

In [0]:
trainer = PPOTrainer(trainer_config, ShapedChainEnv);
for i in range(20):
    print("Training iteration {}...".format(i))
    trainer.train()

env = ShapedChainEnv({})

max_states = []

for i in range(5):
    state = env.reset()
    done = False
    max_state = -1
    cumulative_reward = 0
    while not done:
        action = trainer.compute_action(state)
        state, reward, done, results = env.step(action)
        max_state = max(max_state, state)
        cumulative_reward += reward
    max_states += [max_state]

print("Cumulative reward you've received is: {}!".format(cumulative_reward))
print("Max state you've visited is: {}. This is out of {} states.".format(np.mean(max_state), env.n))
assert (env.n - np.mean(max_state)) / env.n < 0.2, "This policy did not traverse many states."